<a href="https://colab.research.google.com/github/eunji1002/pytorch_example/blob/main/example2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip /content/drive/MyDrive/datasets/archive.zip -d /content/drive/MyDrive/datasets/

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/drive/MyDrive/datasets/train/rings/022.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/023.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/024.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/025.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/026.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/027.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/028.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/029.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/030.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/031.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/032.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/033.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/034.jpg  
  inflating: /content/drive/MyDrive/datasets/train/rings/035.jpg  
  inflating: /content/driv

In [1]:
from torch import nn

class SuperLightMobileNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(SuperLightMobileNet, self).__init__()

        def conv_bn(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True)
            )

        def conv_dw(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, inp, 3, stride, 1, groups=inp, bias=False),
                nn.BatchNorm2d(inp),
                nn.ReLU(inplace=True),

                nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True),
            )
        self.num_classes = num_classes
        self.model = nn.Sequential(
            conv_bn(  3,  16, 2),
            conv_dw( 16,  32, 1),
            conv_dw( 32, 64, 2),
            conv_dw(64, 64, 1),
            conv_dw(64, 128, 2),
            conv_dw(128, 128, 1),
            conv_dw(128, 256, 2),
            conv_dw(256, 256, 1),
            conv_dw(256, 512, 2),
            conv_dw(512, 512, 1),
            conv_dw(512, 1024, 1)
        )
        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(1024, self.num_classes)

    def forward(self, x):
        x = self.model(x)
        x = self.gap(x)
        x = x.view(-1, 1024)
        x = self.fc(x)
        return x

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import torch

#hyp parameters
dataset_path = "/content/drive/MyDrive/datasets"
model_weight_save_path = "/content/drive/MyDrive/Models"
num_classes = 100

batch_size = 40
num_workers = 8
lr = 1e-3

total_epoch = 50

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
import torch
from torch.utils.data import DataLoader
from torch import nn
from torchvision import transforms
import torchvision.datasets as datasets
import os


# Data loading code
traindir = os.path.join(dataset_path, 'train')
testdir = os.path.join(dataset_path, 'test')

# normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                   std=[0.229, 0.224, 0.225])

train_dataset = datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        #normalize,
    ]))

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True,
    num_workers=num_workers, pin_memory=True, drop_last=False)

test_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(testdir, transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        #normalize,
    ])),
    batch_size=batch_size, shuffle=False,
    num_workers=num_workers, pin_memory=True)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [5]:
model = SuperLightMobileNet(num_classes).to(device)

In [6]:
CEloss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [10]:
import numpy as np

total_iteration_per_epoch = int(np.ceil(len(train_dataset)/batch_size))

for epoch in range(1, total_epoch + 1):
    model.train()
    for itereation, (input, target) in enumerate(train_loader):
        images = input.to(device)
        labels = target.to(device)

        # Forward pass
        outputs = model(images)
        loss = CEloss(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print('Epoch [{}/{}], Iteration [{}/{}] Loss: {:.4f}'.format(epoch, total_epoch, itereation+1, total_iteration_per_epoch, loss.item()))
    if epoch % 10 == 0:
      torch.save(model.state_dict(), model_weight_save_path + 'models_' + str(epoch) + ".pth")

    model.eval()
    with torch.no_grad():
      correct = 0
      total = 0
      for input, target in test_loader:
          images = input.to(device)
          labels = target.to(device)

          # Forward pass
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += len(labels)
          correct += (predicted == labels).sum().item()

      print('Epoch [{}/{}], Test Accuracy of the model on the {} test images: {} %'.format(epoch, total_epoch, total, 100 * correct / total))

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Epoch [36/50], Iteration [86/338] Loss: 0.3178
Epoch [36/50], Iteration [87/338] Loss: 0.3667
Epoch [36/50], Iteration [88/338] Loss: 0.2034
Epoch [36/50], Iteration [89/338] Loss: 0.4211
Epoch [36/50], Iteration [90/338] Loss: 0.5804
Epoch [36/50], Iteration [91/338] Loss: 0.5597
Epoch [36/50], Iteration [92/338] Loss: 0.6692
Epoch [36/50], Iteration [93/338] Loss: 0.3638
Epoch [36/50], Iteration [94/338] Loss: 0.4035
Epoch [36/50], Iteration [95/338] Loss: 0.3214
Epoch [36/50], Iteration [96/338] Loss: 0.4566
Epoch [36/50], Iteration [97/338] Loss: 0.2702
Epoch [36/50], Iteration [98/338] Loss: 0.7020
Epoch [36/50], Iteration [99/338] Loss: 0.3019
Epoch [36/50], Iteration [100/338] Loss: 0.2358
Epoch [36/50], Iteration [101/338] Loss: 0.5015
Epoch [36/50], Iteration [102/338] Loss: 0.3855
Epoch [36/50], Iteration [103/338] Loss: 0.2978
Epoch [36/50], Iteration [104/338] Loss: 0.4250
Epoch [36/50], Iteration [105/338] Loss: 0.3474
Epoch [36/50], Ite